In [4]:
'''
 !/usr/bin/python3
 Author: Jonathan Lam Kam Cheung
 Student ID: 040942909
 Date: 19 Nov 2018 at 00:25 AM
 Description: Application analyses text input files to generate a total of words, characters, blank spaces and the number of timeseach words are repeated
              The application accepts only .txt files and exports the analysed data to a .txt file
              The application has 3 frames:
                 -> frame A contains an entry (to manually enter file path or browse the file path)
                                   a reset all button to clean all 3 frames. To be used by user if he wants to re-do a search and clean the results from previous runs
                 -> frame B contains a status bar and a list box. The list box displays all processes ran (each time a .txt file is analysed) 
                                   the user can use the reset all button to clear all frames
                                   when run button is clicked an entry is made to the list box. This is same for completed process and exports which will display the saved file path. 
                                   the list box is sorted from recent to oldest entry
                 -> frame C contains the results of an analysed file. Each runs are recorded with a number and the most recent result is displayed top/first
              The user run 3 different .txt files, one after the another and all results will be stacked into frame C. The export button takes all entries from frame C and writes it to a .txt file
              Each 3 different runs are numbered in sequence
              -> I have made use of website: http://www.charactercountonline.com/ to compare result
              -> I have also used regex to remove the following characters: [,\".!?]
              -> This is because for example Name?, you! or done. is identified as a unique word when Name, you or done are already in the dictionary when splitting
                                
 Open Text Sample
 Import packages/modules
 =============================
'''
try:
    import tkinter as tk
    from tkinter import filedialog as fileDialog
    from tkinter import ttk
    from tkinter import messagebox as messageBox
    import re
except Exception as e:
    messageBox.showerror("Error","An exception occured from when importing the libraries: " + str(e))

'''
Create the application base class
=============================
'''
class myApp():    
    
    ''' Initialize application (attributes, widgets)'''
    def __init__(self, master):
        master.title('My Sample App')
        master.geometry("856x450")
        
        '''
        Add widgets e.g. buttons, labels
        =============================  
        '''
        
        '''Setting the menu bar on top of the window'''
        menuBar = tk.Menu(root)
        fileMenu = tk.Menu(menuBar,tearoff=0)
        fileMenu.add_command(label="Exit", command=self.exitAppClick)  
        menuBar.add_cascade(label="File", menu=fileMenu)        
        exportMenu = tk.Menu(menuBar,tearoff=0)
        exportMenu.add_command(label="Text", command=self.exportTextFile) 
        menuBar.add_cascade(label="Export", menu=exportMenu)
        aboutMenu = tk.Menu(menuBar,tearoff=0)
        aboutMenu.add_command(label="(-_-)",) 
        menuBar.add_cascade(label="About", menu=aboutMenu)
        root.config(menu=menuBar)        

        '''Setting up all 3 frames'''
        self.frameA = tk.Frame(master, borderwidth=2, relief='groove')
        self.frameB = tk.Frame(master, borderwidth=2, relief='groove', background='white')
        self.frameC = tk.Frame(master, relief='groove', background='white')
        self.frameA.grid(row=0, column=0)
        self.frameB.grid(row=0, column=1)
        self.frameC.grid(row=1, column=0, columnspan=2)
        self.frameA.place(x=0, y=0, anchor="nw", width=520, height=200)
        self.frameB.place(x=520, y=0, anchor="nw", width=336, height=200)
        self.frameC.place(x=0, y=200, anchor="nw", width=856, height=250)

        '''Setting up frame A'''
        self.mainLabel = tk.Label(self.frameA, text='Text Analysis - lamk0006')
        self.mainLabel.grid(row=0,column=0,columnspan=3, pady=3)
        self.myStringPath=tk.StringVar(root)        
        self.myEntry = tk.Entry(self.frameA, width=40, textvariable=self.myStringPath)
        self.myEntry.grid(row=1, column=0, padx=25)   
        self.myEntry.config(state="disable")
        self.browseMyFileBtn = tk.Button(self.frameA,text="Browse", command=self.browseMyFileClick, background="#e3e9f4")
        self.browseMyFileBtn.grid(row=1, column=1, padx=10, pady=5, ipadx=15)
        self.clearMyEntry = tk.Button(self.frameA, text="Reset All", command=self.resetAllClick, background="white")
        self.clearMyEntry.grid(row=1, column=2, padx=10, pady=5, ipadx=15)        
        self.runTextAnalysisBtn = tk.Button(self.frameA,text="Run", command=self.runTextAnalysisClick, background="#334353", foreground = "white")
        self.runTextAnalysisBtn.grid(row=3, column=0, columnspan=3, pady=7, padx=10, ipadx=30)
        self.runTextAnalysisBtn.config(state="disable")      
        self.checkBox = tk.BooleanVar()
        self.enterPathManuallyBtn = tk.Checkbutton(self.frameA,text = "Enter File Path Manually",variable = self.checkBox,command = self.toggleManualFilePath)
        self.enterPathManuallyBtn.grid(row=2,column=0, columnspan=3, sticky="w", padx=20)
        
        '''Setting up frame B'''        
        self.statusLabel = tk.Label(self.frameB, text="Status: ", background="white")
        self.statusLabel.grid(row=0,column=0, pady=2, sticky='w')
        self.myProgressBar = ttk.Progressbar(self.frameB, orient="horizontal", length=260)
        self.myProgressBar.grid(row=1, column=0, pady=1) 
        self.statusListBox = tk.Listbox(self.frameB, width=54, height=7, borderwidth=0)
        self.statusListBox.grid(row=2,column=0, pady=7, padx=2)   
        self.runClickCount = 0;        
        self.statusListBoxScrollBar = tk.Scrollbar(self.frameB, command=self.statusListBox.xview, orient='horizontal')
        self.statusListBox.configure(xscrollcommand=self.statusListBoxScrollBar.set)
        self.statusListBoxScrollBar.grid(row=3, column=0, columnspan=2, sticky="ew")  
        
        '''Setting up frame C'''        
        self.resultText = tk.Text(self.frameC, width=104, height=15)
        self.resultTextScrollBar = tk.Scrollbar(self.frameC, command=self.resultText.yview)
        self.resultText.configure(yscrollcommand=self.resultTextScrollBar.set)
        self.resultText.grid(row=0, column=0, sticky="nsew")
        self.resultTextScrollBar.grid(row=0, column=1, rowspan=2, sticky="ns")   
       
    ''' 
    Add Methods (order does not matter)
    =============================
    '''  
    
    '''checking if checkbox is clicked to change setup'''
    def toggleManualFilePath(self):
        try:
            checkBoxVal = self.checkBox.get()
            if checkBoxVal:
                self.myEntry.config(state="normal")
                self.browseMyFileBtn.config(state="disable")
                self.runTextAnalysisBtn.config(state='normal')
            else:
                self.myEntry.delete(0,"end") 
                self.myEntry.config(state="disable")
                self.browseMyFileBtn.config(state="normal")
                self.runTextAnalysisBtn.config(state='disable')                
        except Exception as e:
            messageBox.showerror("Manual File Path Error","An exception occurred in when toggling the Manual File Path: \n" + str(e))            
        else:
            self.myEntry.delete(0,"end")           
    
    '''function to export file to text'''
    def exportTextFile(self):
        self.retrieveText('text')
            
    '''function to export file to text'''    
    def retrieveText(self,paramExport):
        try:
            '''retrieve all lines from the textbox from frame C, if the user ran 3 different files without the Clear previous results, then
            the textbox is filled in with all 3 text analysis, each numbered with a number to identify each of them'''
            text2Save = str(self.resultText.get('1.0', tk.END))
            
            '''check if textbox is empty, then pop up a message to the user notifying him'''
            if text2Save == "\n":
                messageBox.showerror("Export?","There is nothing to export. Please analyse a .txt file first")          
                return
            
            '''if export is of text type, defaultextension is .txt'''
            if paramExport == 'text':
                defaultFileName = ""
                if self.runClickCount == 1:
                    defaultFileName = self.truncFileName.replace(".txt", "") + "_analysis"
                saveFilePath = fileDialog.asksaveasfile(initialfile = defaultFileName, mode='w', title="Save the file", defaultextension=".txt")
            if saveFilePath is None:
                return         
            
            '''writing to the destination file'''
            saveFilePath.write(text2Save)
            saveFilePath.close()
        
            '''update the listbox from frame B'''
            nameSaveFilePath = saveFilePath.name
            getSaveFileName = nameSaveFilePath.split("/")
            truncSaveFileName = getSaveFileName[-1] 
        except Exception as e:
            messageBox.showerror("Export Error","An exception occurred when trying to export the file: \n" + str(e))
        except FileNotFoundError:
            messageBox.showerror("Export Error","An exception occurred, the file path is not found")
        else:
            self.statusListBox.insert("0","0. " + nameSaveFilePath)
            messageBox.showinfo("Exported!","The text file has been exported to \n" + nameSaveFilePath)        
            self.statusListBox.insert("0","0. " + truncSaveFileName + " - " + "Exported")  
            
    '''function to wrap the clearing of all entries'''        
    def resetAllClick(self):
        try:
            userAnswer = messageBox.askyesno("Proceed?","All frames will be erased and set to default. Do you want to continue?")
            if userAnswer:
                self.processReset()
        except Exception as e:
            messageBox.showerror("Reset Error","An exception occurred when trying to clear/reset all values: \n" + str(e))
           
    '''function to clear all entries'''
    def processReset(self):
        self.myEntry.config(state="normal")                
        self.myEntry.delete(0,"end")
        self.runTextAnalysisBtn.config(state="disable")
        self.resultText.delete("1.0",tk.END) 
        self.statusListBox.delete(0,tk.END)
        self.runClickCount = 0
        self.checkBox.set(0)
        self.toggleManualFilePath()
        self.myProgressBar["value"] = 0   
         
    '''function to browse the file'''    
    def browseMyFileClick(self):
        try:
            self.myEntry.config(state="normal")            
            self.myEntry.delete(0, "end")
            try:
                self.filePath = str(fileDialog.askopenfile(parent=root,mode='rb',title='Choose a file'))
                if not self.hasFilePath():  
                    self.myEntry.config(state="disable")                     
                    return   
            except Exception as e:
                messageBox.showerror("Browse Error","An exception occurred when browsing the file_1: \n" + str(e))
            else:
                dupFilePath = self.filePath
                self.splitFilePath = dupFilePath.split("'")
                self.myEntry.insert(0,self.splitFilePath[1])          
        except Exception as e:
            messageBox.showerror("Browse Error","An exception occurred in when browsing the file_2: \n" + str(e))
        else:
            self.runTextAnalysisBtn.config(state='normal')  
            self.myEntry.config(state='disable')                 
            
    '''function to exit application'''
    def exitAppClick(self):
        try:
            root.destroy()
        except Exception as e:
            messageBox.showerror("Exit Error","An exception occurred when exiting the app: \n" + str(e))
            
    '''function to update progress bar'''        
    def updateProgressBar(self,percentRemainingLines):
        try:
            self.myProgressBar["value"] += percentRemainingLines  
        except Exception as e:
            messageBox.showerror("Progress Bar Error","An exception occurred when updating the progression bar: \n" + str(e))
        else:
            self.myProgressBar.update_idletasks()  
          
    '''function to get number of lines in the text file to analyse'''   
    def getNoOfLines(self,txtFileStr):
        with open(txtFileStr,'r') as countTxtFileLines:
            try:
                for lineNo, line in enumerate(countTxtFileLines, 1):
                    pass
                return lineNo + 1
            except Exception as e:
                messageBox.showerror("File No. of Lines Error","An exception occurred when trying to get the no. of lines in the file: \n" + str(e))
            
    '''function to check if the file path is not empty'''
    def hasFilePath(self):        
        if (len(self.filePath) == 0) or (self.filePath == "None"):
            return False
        return True
    
    '''function to check if the file path is not empty'''
    def hasEntryFilePath(self):        
        self.myEntryPath = self.myStringPath.get()
        if (len(self.myEntryPath) == 0) or (self.myEntryPath == "None"):
            return False
        return True   
            
    '''function to pop up user confirmation'''        
    def userPromptToConfirm(self):  
        try:
            '''spliting to get the filename so that the user can confirm when he is prompted with a dialog'''
            countForwardSlash = self.myEntryPath.count("/")
            countBackSlash = self.myEntryPath.count("\\")
            strSplitPath = self.myEntryPath
            strGetFileName = ''
            if countForwardSlash > 0:
                strGetFileName = strSplitPath.split("/")
            elif countBackSlash > 0:
                strGetFileName = strSplitPath.split("\\")
            else: 
                messageBox.showerror("File Error","Please enter a correct file path \nFor example: D:/Users/Jonathan/Desktop/X.txt")
                return
            self.getFileName = strGetFileName
            self.truncFileName = self.getFileName[-1]
        except Exception as e:
            messageBox.showerror("Confirm Error","An exception occurred in the file path: " + str(e))
        else:
            userAnswer = messageBox.askyesno("Proceed?", "Do you want to proceed by analysing file: \n" + self.truncFileName , icon='warning')
            return userAnswer
            
    '''function to update frame C by inserting from bottom to top'''
    def updateResultText(self,fName,wrds,chars,blkSpace,wrdDic):
        try:
            '''check if word dictionary is not empty'''
            if not (wrdDic == []):
                for eWord, eCount in wrdDic.items():
                    self.resultText.insert('1.0',"  " +self.convToString(eWord) + " : " + self.convToString(eCount) + "\n")  
                self.resultText.insert('1.0',"  Word/Count Dictionary: \n")                 
            self.resultText.insert('1.0',"  Total Word Count: " + self.convToString(wrds) + "\n")          
            '''calculate the total percentage of blank char spaces against total chars'''        
            percentBlkSpaceVChars = '{0:.2f}'.format((blkSpace/chars)*100)
            self.resultText.insert('1.0',"  Percentage Blank Character: " + self.convToString(percentBlkSpaceVChars) + "% \n")         
            self.resultText.insert('1.0',"  Total Character Count: " + self.convToString(chars) + "\n")   
            self.resultText.insert('1.0',"  Total Blank Character Count: " + self.convToString(blkSpace) + "\n")
            self.resultText.insert('1.0',"  Total Non-blank Character Count: " + self.convToString(chars-blkSpace) + "\n")        
            self.resultText.insert('1.0',str(self.runClickCount) + "." + "Name of text: " + fName + "\n")  
        except ZeroDivisionError:
            messageBox.showerror("Result Error","An exception occurred when publishing the result: Division by Zero")
        except Exception as e:
            messageBox.showerror("Result Error","An exception occurred when publishing the result: \n" + str(e))
             
    '''own function to convert to string, same as str function'''
    def convToString(self,paraTxt):
        return str(paraTxt)          
        
    '''function to run text analysis of the browsed file'''   
    def runTextAnalysisClick(self):
        try:
            '''check if file path is not empty'''
            if not self.hasEntryFilePath():
                messageBox.showerror("File Error","File path is empty")            
                return            
            '''check file extension to be exclusively .txt'''
            if not self.myEntry.get().endswith('.txt'):
                messageBox.showerror("File Error","File format is not supported. Please use .txt files only.")
                return    
            '''asks for user to confirm'''        
            if not self.userPromptToConfirm():
                return                
            self.myProgressBar["value"] = 0 
            noOfLines = self.getNoOfLines(self.myEntryPath)
        except FileNotFoundError:
            messageBox.showerror("Run Error","An exception occurred, the file is not found")            
        except Exception as e:
            messageBox.showerror("Run Error","An exception occurred when trying to analyse the file: \n" + str(e))
        else:
            '''start the reading of the browsed file'''
            with open(self.myEntryPath,'r') as self.txtFile: 
                try:
                    '''set all local variables to be used for the different requirements'''
                    noOfWords = 0
                    noOfCharacters = 0
                    noOfBlankSpace = 0            
                    wordsDict = dict()                    
                    '''variable to set a count on the number of times the run button is clicked. This is used to place a number to each Run analysis'''
                    self.runClickCount += 1
                    self.statusListBox.insert("0",str(self.runClickCount) + ". " + self.truncFileName + " - " + "Processing...")                     
                    '''declare another class to clean the text file'''
                    myCleanFile = rmFileClass()                           
                    for lineNo, line in enumerate(self.txtFile, 1):  
                        '''update value to be set in the progress bar'''
                        self.updateProgressBar((lineNo / noOfLines)*200)                        
                        '''remove all change of line which adds a blankspace and character count to the totals''' 
                        '''made use of website: http://www.charactercountonline.com/ to compare result'''
                        lineStrip = myCleanFile.cleanNewLine(line)                         
                        '''remove some symbols which are not of use'''
                        regexLine = myCleanFile.cleanChar(lineStrip)                        
                        splitLine = regexLine.split()
                        noOfWords += len(splitLine)                
                        for wordsInLine in splitLine:
                            if wordsInLine in wordsDict:
                                wordsDict[wordsInLine] += 1
                            else:
                                wordsDict[wordsInLine] = 1          
                        for letter in lineStrip:            
                            noOfCharacters += 1 
                            '''check if the char is a space'''                         
                            if (letter.isspace()):
                                noOfBlankSpace += 1                                     
                except ZeroDivisionError:
                    messageBox.showerror("Run Error","An exception occurred when calculating the totals: Division by Zero")
                    self.processReset();  
                except FileNotFoundError:
                    messageBox.showerror("Run Error","An exception occurred, the file is not found")
                    self.processReset();                    
                except Exception as e:
                    messageBox.showerror("Run Error","An exception occurred when calculation the totals: \n" + str(e))
                    self.processReset();                    
                else:                    
                    '''reset the progress bar'''
                    self.myProgressBar["value"] = 0                      
                    '''update the textbox in frame C with the updated results. Made use of a function and parameters'''
                    self.updateResultText(self.truncFileName,noOfWords,noOfCharacters,noOfBlankSpace,wordsDict)                         
                    '''update listbox from frame B, indicating that the process is complete'''            
                    self.statusListBox.insert("0",str(self.runClickCount) + ". " + self.truncFileName + " - " + "Completed")                             

'''class to be used to clean specific requirements'''
class rmFileClass():
    
    '''removing new line which adds on to the total of char for nothing'''
    def cleanNewLine(self,oline):
        return oline.rstrip('\n')
    
    '''removing useless chars'''
    def cleanChar(self,oline):
        regex = re.compile('[,\".!?]')
        return regex.sub('',oline)
        

''' 
Create the application
=============================
'''
root = tk.Tk()   
app = myApp(root)

''' 
Start the application
=============================
'''
root.mainloop()